In [1]:
from langgraph.graph import StateGraph, START, END
#import google chat model
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [4]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluation: str

In [5]:
def create_outline(state: BlogState) -> BlogState:

    title = state['title']
    prompt = f'Generate a detailed outline for a blog on the - {title}'
    outline = model.invoke(prompt).content
    state['outline'] = outline

    return state


In [6]:
def create_blog(state: BlogState) -> BlogState:
    
    title = state['title']
    outline = state['outline']
    prompt = f'Write a detailed blog the title - {title} using the following outline \n {outline}'
    content = model.invoke(prompt).content
    state['content'] = content
    
    return state


In [7]:
def evaluate_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    content = state['content']
    prompt = (
        "You are a concise blog reviewer. Rate the blog content 1-10 based on how well it "
        "follows the outline while staying relevant to the title. Provide a one-line justification.\n"
        f"Title: {title}\n"
        f"Outline: {outline}\n"
        f"Blog: {content}"
    )
    state['evaluation'] = model.invoke(prompt).content
    return state


In [8]:
graph = StateGraph(BlogState)

# nodes 
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)

# Edges

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow = graph.compile()

In [ ]:
intial_state = {'title': 'Rise of AI in Pakistan'}
final_state = workflow.invoke(intial_state)
print(final_state)

In [10]:
print(final_state['outline'])

NameError: name 'final_state' is not defined

In [11]:
print(final_state['content'])

NameError: name 'final_state' is not defined

In [13]:
print(final_state['evaluation'])

**Rating:** 10/10

**Justification:** The blog meticulously follows the comprehensive outline, addressing every section and sub-point with relevant content and a consistent, appropriate tone.
